In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import lognorm
from itertools import chain, combinations

In [2]:
plt.style.use('ggplot')

In [3]:
# tip distribution
tips = lognorm(s=.9, scale=2)

In [4]:
# coin prices
b_P = 38000
e_P = 2638

In [5]:
# liquidity pool quantities
be_Q = 2714.78
eb_Q = 50826.90
ub_Q = 22847370.15
bu_Q = 1874.21
ue_Q = 64330753.58
eu_Q = 52702.75

In [6]:
# starting quantities
b_start = 1
e_start = 14.4
u_start = 38000

In [7]:
# read in data
BTC_ETH = np.loadtxt('data/BTC_ETH.csv')
USDC_BTC = np.loadtxt('data/USDC_BTC.csv')
USDC_ETH = np.loadtxt('data/USDC_ETH.csv')

In [8]:
# quantity distribution
b_e = lognorm(*lognorm.fit(BTC_ETH))
u_b = lognorm(*lognorm.fit(USDC_BTC))
u_e = lognorm(*lognorm.fit(USDC_ETH))

In [9]:
TOKENS = ['BCT', 'ETH', 'USDC']
QUANTITIES = [b_e, u_b, u_e]
PRICE = [38000, 2638, 1]

In [10]:
def get_trades(N):
    # initialize
    T = pd.DataFrame(columns=('quantity', 'sell_token', 'buy_token', 'tip', 'index'))
    
    # add a trade one at a time
    for i in range(N):
        sell_idx, buy_idx = np.random.choice([0, 1, 2], 2, replace=False)
        c_sell, c_buy = TOKENS[sell_idx], TOKENS[buy_idx]
        qnty = QUANTITIES[sell_idx + buy_idx - 1].rvs() / PRICE[sell_idx]
        tip = tips.rvs()
        
        T.loc[i] = [qnty, c_sell, c_buy, tip, i]
        
    return T

In [11]:
get_trades(120)

,quantity,sell_token,buy_token,tip,index
0,4.274998e-04,ETH,USDC,1.244919,0
1,1.774503e+01,ETH,USDC,4.488857,1
2,1.985577e+08,USDC,ETH,0.406713,2
3,2.974004e+00,ETH,USDC,0.476585,3
4,2.000002e-05,BCT,ETH,10.448880,4
...,...,...,...,...,...
115,3.418493e-02,ETH,BCT,2.248392,115
116,2.000560e-05,BCT,ETH,6.810484,116
117,7.324756e-01,BCT,ETH,5.807682,117
118,2.357632e-03,BCT,USDC,1.120550,118


In [12]:
def get_trades(N):
    # get quantities
    be_Q = b_e.rvs(N)/b_P
    eb_Q = b_e.rvs(N)/e_P
    ub_Q = u_b.rvs(N)
    bu_Q = u_b.rvs(N)/b_P
    ue_Q = u_e.rvs(N)
    eu_Q = u_e.rvs(N)/e_P
    Q = np.concatenate((be_Q, eb_Q, ub_Q, bu_Q, ue_Q, eu_Q))
    
    # get tips
    t = tips.rvs(6*N)
    
    # get token names
    b_temp = np.full(N, 'BTC')
    e_temp = np.full(N, 'ETH')
    u_temp = np.full(N, 'USDC')
    c_sell = np.concatenate((b_temp, e_temp, u_temp, b_temp, u_temp, e_temp))
    c_buy = np.concatenate((e_temp, b_temp, b_temp, u_temp, e_temp, u_temp))
    
    # array of trades
    df =  pd.DataFrame(
        np.transpose([Q, c_sell, c_buy, t]),
        columns=('quantity', 'sell_token', 'buy_token', 'tip')
    )
    df = df.astype({'quantity':float, 'tip':float})
    df['idx'] = df.index
    return df

In [13]:
get_trades(20)

,quantity,sell_token,buy_token,tip,idx
0,0.453933,BTC,ETH,0.872123,0
1,0.000021,BTC,ETH,1.288540,1
2,0.153575,BTC,ETH,2.621163,2
3,0.000020,BTC,ETH,0.878414,3
4,0.972173,BTC,ETH,0.650224,4
...,...,...,...,...,...
115,0.002525,ETH,USDC,0.262733,115
116,22046.204368,ETH,USDC,3.976194,116
117,0.692314,ETH,USDC,1.793356,117
118,0.000453,ETH,USDC,0.893839,118


In [11]:
T = get_trades(15)
T

,quantity,sell_token,buy_token,tip,idx
0,0.018570,BTC,ETH,1.375598,0
1,0.000020,BTC,ETH,3.502722,1
2,0.000236,BTC,ETH,2.007601,2
3,0.114037,BTC,ETH,10.126443,3
4,0.567657,BTC,ETH,2.825785,4
...,...,...,...,...,...
85,0.001317,ETH,USDC,2.731851,85
86,0.000583,ETH,USDC,1.461969,86
87,0.000379,ETH,USDC,7.487582,87
88,0.000416,ETH,USDC,2.640764,88


In [12]:
G_eb = T[(T['sell_token'] == 'ETH') & (T['buy_token'] == 'BTC')].values
g_eb = list(chain.from_iterable(combinations(G_eb, r) for r in range(1, len(G_eb)+1)))
G_be = T[(T['sell_token'] == 'BTC') & (T['buy_token'] == 'ETH')].values
g_be = list(chain.from_iterable(combinations(G_be, r) for r in range(1, len(G_be)+1)))
G_bu = T[(T['sell_token'] == 'BTC') & (T['buy_token'] == 'USDC')].values
g_bu = list(chain.from_iterable(combinations(G_bu, r) for r in range(1, len(G_bu)+1)))
G_ub = T[(T['sell_token'] == 'USDC') & (T['buy_token'] == 'BTC')].values
g_ub = list(chain.from_iterable(combinations(G_ub, r) for r in range(1, len(G_ub)+1)))
G_eu = T[(T['sell_token'] == 'ETH') & (T['buy_token'] == 'USDC')].values
g_eu = list(chain.from_iterable(combinations(G_eu, r) for r in range(1, len(G_eu)+1)))
G_ue = T[(T['sell_token'] == 'USDC') & (T['buy_token'] == 'ETH')].values
g_ue = list(chain.from_iterable(combinations(G_ue, r) for r in range(1, len(G_ue)+1)))
T_hat = g_eb + g_be + g_bu + g_ub + g_eu + g_ue

In [13]:
P = lambda a, N, A: (a*N*(a + 2*A + N)) / (a**2 + 2*a*A + a*N + A**2)

In [ ]:
APPT = pd.DataFrame(columns=['sell_token', 'buy_token', 'profit', 'weight', 'appt'])
trade_dict = {}
for i, g in enumerate(T_hat):
    # get coin names, starting investment, and pool size
    c_sell, c_buy = g[0][1], g[0][2]
    if c_sell == 'BTC':
        a0 = b_start
        if c_buy == 'ETH':
            A0 = be_Q
        elif c_buy == 'USDC':
            A0 = bu_Q
    elif c_sell == 'ETH':
        a0 = e_start
        if c_buy == 'BTC':
            A0 = eb_Q
        elif c_buy == 'USDC':
            A0 = eu_Q
    elif c_sell == 'USDC':
        a0 = u_start
        if c_buy == 'ETH':
            A0 = ue_Q
        elif c_buy == 'BTC':
            A0 = ub_Q
    
    # add to dataframe
    N = sum([g_i[0] for g_i in g])
    profit = P(a0, N, A0)
    tip_sum = sum([g_i[3] for g_i in g])
    profit += tip_sum
    weight = len(g) + 2
    appt = profit / weight
    APPT.loc[len(APPT)] = [c_sell, c_buy, profit, weight, appt]
    
    # add to dictionary
    trade_dict[i] = {g_i[4] for g_i in g}

In [49]:
APPT

,sell_token,buy_token,profit,weight,appt
0,ETH,BTC,4.892563e-01,3,1.630854e-01
1,ETH,BTC,5.251452e-01,3,1.750484e-01
2,ETH,BTC,3.158056e+05,3,1.052685e+05
3,ETH,BTC,3.707453e+00,3,1.235818e+00
4,ETH,BTC,8.665465e-01,3,2.888488e-01
...,...,...,...,...,...
6133,USDC,ETH,3.665725e+08,11,3.332477e+07
6134,USDC,ETH,1.276580e+05,11,1.160527e+04
6135,USDC,ETH,3.665725e+08,11,3.332477e+07
6136,USDC,ETH,3.665725e+08,11,3.332477e+07


In [137]:
APPT.loc[4091, 'profit']

40910026631.55356

In [50]:
trade_in_block = np.full(len(T), False)
group_in_block = np.full(len(APPT), False)

In [62]:
def adjust_group(k):
    # initialize
    new_dict = {}
    group = trade_dict[k]
    k_sell = APPT.loc[k, 'sell_token']
    k_buy = APPT.loc[k, 'buy_token']
    k_profit = APPT.loc[k, 'profit']
    k_weight = APPT.loc[k, 'weight']
    
    # add to block
    group_in_block[k] = True
    APPT.drop(k, inplace=True)
    trade_dict.pop(k)
    
    # adjust/remove groups
    for key, value in trade_dict.items():
        # adjust supersets
        if group.issubset(value):
            APPT.loc[key, 'profit'] = APPT.loc[key, 'profit'] - k_profit
            APPT.loc[key, 'weight'] = APPT.loc[key, 'weight'] - k_weight
            APPT.loc[key, 'appt'] = APPT.loc[key, 'profit'] / APPT.loc[key, 'weight']
            new_dict[key] = value
        else:
            # remove nonsupersets
            if (k_sell == APPT.loc[key, 'sell_token']) & (k_buy == APPT.loc[key, 'buy_token']):
                APPT.drop(key, inplace=True)
            else:
                new_dict[key] = value
            
    # remove single trades
    T.drop(group, inplace=True)
            
    return new_dict

In [77]:
def adjust_trade(k):
    # initialize
    new_dict = {}
    k_sell = T.loc[k, 'sell_token']
    k_buy = T.loc[k, 'buy_token']
    k_profit = T.loc[k, 'tip']
    k_weight = 1
    
    # add to block
    trade_in_block[k] = True
    T.drop(k, inplace=True)
    
    # adjust/remove groups
    for key, value in trade_dict.items():
        # adjust supersets
        if k in value:
            APPT.loc[key, 'profit'] = APPT.loc[key, 'profit'] - k_profit
            APPT.loc[key, 'weight'] = APPT.loc[key, 'weight'] - k_weight
            APPT.loc[key, 'appt'] = APPT.loc[key, 'profit'] / APPT.loc[key, 'weight']
            new_dict[key] = value
        else:
            # remove nonsupersets
            if (k_sell == APPT.loc[key, 'sell_token']) & (k_buy == APPT.loc[key, 'buy_token']):
                APPT.drop(key, inplace=True)
            else:
                new_dict[key] = value
            
    return new_dict

In [57]:
k = 6137

trade_dict = adjust_group(k)

In [61]:
APPT

,sell_token,buy_token,profit,weight,appt
0,ETH,BTC,0.489256,3,0.163085
1,ETH,BTC,0.525145,3,0.175048
2,ETH,BTC,315805.635358,3,105268.545119
3,ETH,BTC,3.707453,3,1.235818
4,ETH,BTC,0.866546,3,0.288849
...,...,...,...,...,...
5110,ETH,USDC,196.601025,11,17.872820
5111,ETH,USDC,193.595705,11,17.599610
5112,ETH,USDC,196.754260,11,17.886751
5113,ETH,USDC,197.559039,11,17.959913


In [63]:
k = 136

trade_dict = adjust_group(k)

{16, 19, 12}

{10}
{11}
{12}
{13}
{14}
{15}
{16}
{17}
{18}
{19}
{10, 11}
{10, 12}
{10, 13}
{10, 14}
{10, 15}
{16, 10}
{17, 10}
{10, 18}
{10, 19}
{11, 12}
{11, 13}
{11, 14}
{11, 15}
{16, 11}
{17, 11}
{18, 11}
{19, 11}
{12, 13}
{12, 14}
{12, 15}
{16, 12}
{17, 12}
{18, 12}
{19, 12}
{13, 14}
{13, 15}
{16, 13}
{17, 13}
{18, 13}
{19, 13}
{14, 15}
{16, 14}
{17, 14}
{18, 14}
{19, 14}
{16, 15}
{17, 15}
{18, 15}
{19, 15}
{16, 17}
{16, 18}
{16, 19}
{17, 18}
{17, 19}
{18, 19}
{10, 11, 12}
{10, 11, 13}
{10, 11, 14}
{10, 11, 15}
{16, 10, 11}
{17, 10, 11}
{18, 10, 11}
{19, 10, 11}
{10, 12, 13}
{10, 12, 14}
{10, 12, 15}
{16, 10, 12}
{17, 10, 12}
{10, 18, 12}
{10, 19, 12}
{10, 13, 14}
{10, 13, 15}
{16, 10, 13}
{17, 10, 13}
{10, 18, 13}
{10, 19, 13}
{10, 14, 15}
{16, 10, 14}
{17, 10, 14}
{10, 18, 14}
{10, 19, 14}
{16, 10, 15}
{17, 10, 15}
{10, 18, 15}
{10, 19, 15}
{16, 17, 10}
{16, 10, 18}
{16, 10, 19}
{17, 10, 18}
{17, 10, 19}
{19, 10, 18}
{11, 12, 13}
{11, 12, 14}
{11, 12, 15}
{16, 11, 12}
{17, 11, 12

In [64]:
APPT

,sell_token,buy_token,profit,weight,appt
220,ETH,BTC,0.510969,1,0.510969
276,ETH,BTC,0.526287,1,0.526287
326,ETH,BTC,3.707946,1,3.707946
336,ETH,BTC,0.866576,1,0.866576
342,ETH,BTC,0.474925,1,0.474925
...,...,...,...,...,...
5110,ETH,USDC,196.601025,11,17.872820
5111,ETH,USDC,193.595705,11,17.599610
5112,ETH,USDC,196.754260,11,17.886751
5113,ETH,USDC,197.559039,11,17.959913


In [75]:
T

,quantity,sell_token,buy_token,tip,idx
0,2.783788e-05,BTC,ETH,3.064189,0
1,2.132431e-02,BTC,ETH,2.805667,1
2,2.128208e-05,BTC,ETH,0.427271,2
3,1.060918e-02,BTC,ETH,1.199622,3
4,2.735918e+01,BTC,ETH,1.002327,4
5,2.000000e-05,BTC,ETH,5.900794,5
6,2.000207e-05,BTC,ETH,1.018700,6
7,2.646393e-04,BTC,ETH,9.964691,7
8,4.057054e-04,BTC,ETH,0.744322,8
9,2.848110e-03,BTC,ETH,7.350319,9


In [78]:
trade_dict = adjust_trade(30)

In [144]:
T_hat[20]

(array([0.014282699523288858, 'ETH', 'BTC', 0.5251371464871706, 11],
       dtype=object),
 array([0.00617934469362585, 'ETH', 'BTC', 3.707449104063335, 13],
       dtype=object))

In [80]:
APPT

,sell_token,buy_token,profit,weight,appt
220,ETH,BTC,0.510969,1,0.510969
276,ETH,BTC,0.526287,1,0.526287
326,ETH,BTC,3.707946,1,3.707946
336,ETH,BTC,0.866576,1,0.866576
342,ETH,BTC,0.474925,1,0.474925
...,...,...,...,...,...
5110,ETH,USDC,196.601025,11,17.872820
5111,ETH,USDC,193.595705,11,17.599610
5112,ETH,USDC,196.754260,11,17.886751
5113,ETH,USDC,197.559039,11,17.959913


In [81]:
trade_dict = adjust_trade(0)

In [82]:
T

,quantity,sell_token,buy_token,tip,idx
1,2.132431e-02,BTC,ETH,2.805667,1
2,2.128208e-05,BTC,ETH,0.427271,2
3,1.060918e-02,BTC,ETH,1.199622,3
4,2.735918e+01,BTC,ETH,1.002327,4
5,2.000000e-05,BTC,ETH,5.900794,5
6,2.000207e-05,BTC,ETH,1.018700,6
7,2.646393e-04,BTC,ETH,9.964691,7
8,4.057054e-04,BTC,ETH,0.744322,8
9,2.848110e-03,BTC,ETH,7.350319,9
10,2.717321e-01,ETH,BTC,0.489102,10


In [83]:
APPT

,sell_token,buy_token,profit,weight,appt
220,ETH,BTC,0.510969,1,0.510969
276,ETH,BTC,0.526287,1,0.526287
326,ETH,BTC,3.707946,1,3.707946
336,ETH,BTC,0.866576,1,0.866576
342,ETH,BTC,0.474925,1,0.474925
...,...,...,...,...,...
5110,ETH,USDC,196.601025,11,17.872820
5111,ETH,USDC,193.595705,11,17.599610
5112,ETH,USDC,196.754260,11,17.886751
5113,ETH,USDC,197.559039,11,17.959913


In [142]:
T.nlargest(0, 'tip').sum()

quantity      0.0
sell_token    0.0
buy_token     0.0
tip           0.0
dtype: float64